In [1]:
!pip install scipy

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


In [2]:
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
import numpy as np

In [3]:
class lstm(nn.Module):
    def __init__(self,num):
        super().__init__()
        self.lstm = nn.LSTM(1,20,2,dropout=0.1,batch_first=True)
        self.linear = nn.Linear(20,5)
        self.relu=nn.ReLU()
        self.linear1 = nn.Linear(5,1)
        self.num = num
        self.lstm1 = nn.LSTM(1,20,2,dropout=0.2,batch_first=True)
    def forward(self,x):
        #print(x.shape)
        x = x.unsqueeze(2)
        output, (hn, cn) = self.lstm(x)
        output1=[]
        
        for i in range(self.num):
            output, (hn, cn) = self.lstm1(x[:,-1,:].unsqueeze(1),(hn,cn))
            out = self.relu(self.linear(output[:,-1,:]))
            out1 = self.linear1(out)
            if i <=self.num-1:
                out1 = out1.unsqueeze(2)
                output1.append(out1)
                #print(out1.shape)
                x = torch.cat([x,out1],dim=1)
                output, (hn, cn) = self.lstm(x[:,1:,:])
        return torch.cat(output1,dim=0)

In [4]:
model = lstm(2)
model.to('cpu')

lstm(
  (lstm): LSTM(1, 20, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=20, out_features=5, bias=True)
  (relu): ReLU()
  (linear1): Linear(in_features=5, out_features=1, bias=True)
  (lstm1): LSTM(1, 20, num_layers=2, batch_first=True, dropout=0.2)
)

In [5]:
loadmodel = torch.load('forward_model.pt',map_location='cpu')
model.load_state_dict(loadmodel)

<All keys matched successfully>

In [6]:
data = pd.read_csv('result_eva4.csv')

In [7]:
data = data[data['name']=='cluster']

In [8]:
data.to_csv('lstmdata.csv')

In [9]:
data_time = data['time']
data_time = data_time[:42]
data_time = data_time[::-1]
data_time

88    2022-04-12
87    2022-04-13
86    2022-04-14
85    2022-04-15
84    2022-04-16
83    2022-04-17
82    2022-04-19
81    2022-04-20
80    2022-04-25
79    2022-04-26
78    2022-04-27
77    2022-04-30
76    2022-05-01
75    2022-05-02
74    2022-05-04
73    2022-05-05
72    2022-05-06
71    2022-05-07
70    2022-05-08
69    2022-05-09
68    2022-05-11
67    2022-05-12
66    2022-05-13
65    2022-05-17
64    2022-05-18
63    2022-05-19
62    2022-05-22
61    2022-05-23
60    2022-05-24
59    2022-05-25
58    2022-05-26
57    2022-05-27
56    2022-06-05
55    2022-06-06
54    2022-06-16
53    2022-06-17
52    2022-06-18
51    2022-06-19
50    2022-06-20
49    2022-06-21
48    2022-06-22
47    2022-06-23
Name: time, dtype: object

In [10]:
data_x = data['dcp']

In [11]:
data_x = data_x[:42]

In [12]:
data_x = data_x.to_numpy()
data_x = data_x[::-1]

In [13]:
input_x=data_x[-5:]
input_x = torch.from_numpy(input_x.copy())
input_x = input_x.reshape(1,5).float()

In [14]:
input_x

tensor([[180., 298., 306., 234.,  12.]])

In [15]:
output = model(input_x)
output = torch.flatten(output)

In [16]:
output = output.detach().numpy()


In [17]:
data_x = np.concatenate([data_x,output])
data_x

array([383.52830189, 264.        , 290.        , 206.        ,
       508.        , 127.8125    ,  70.        , 214.        ,
        82.        , 386.        ,  90.        , 136.        ,
       312.        , 102.        , 244.        , 262.        ,
       204.        , 326.        , 332.        ,  66.        ,
       251.44186047, 334.        , 356.19631902, 210.        ,
       306.        ,  52.        ,  82.        , 300.02758621,
       294.        , 250.        , 270.        , 306.        ,
       992.        , 626.        , 212.        , 268.        ,
       248.        , 180.        , 298.        , 306.        ,
       234.        ,  12.        , 427.81283569, 427.81283569])

In [18]:
def np_move_avg(a,n,mode="same"):
    return(np.convolve(a, np.ones((n,))/n, mode=mode))

In [19]:
def move_avg(a,n,mode="same"):
    return(np.convolve(a, np.ones((n,))/(n+1), mode=mode))

In [20]:
def compute_fj(x,window):
    out = np_move_avg(x,window)
    out1 = x-out
    #length = len(x)//7
    out1 =out1- np.median(out1)
    return out1
    

In [21]:
def compute_srbo(x):
    fft_y = scipy.fft.fft(x)
    abs_y=np.abs(fft_y)                # 取复数的绝对值，即复数的模(双边频谱)
    angle_y=np.angle(fft_y) 
    print(abs_y)
    l=np.log2(abs_y+0.001)
    kernel = np.ones(3)
    h = kernel/3.
    print(h)
    hq=np.convolve(l, h, mode='same')
    r = l-hq
    out_put = r+1j*angle_y
    output = np.abs(scipy.fft.ifft(out_put))
    return output

In [22]:
def compute_d(x,window):
    out = move_avg(x,window)
    d = (x-out)/out
    return d

In [23]:
window = 7

In [24]:
out = compute_fj(data_x,7)

In [25]:
output = compute_srbo(out)

[ 864.50344059  592.50505948  478.07673821  912.98132691 1258.55725821
  545.3760869   655.20455642  462.55791801 2181.63500219 2163.24536665
  318.73741088 1276.5770816  1096.59396683  943.92288488  706.03830059
 1034.06483197  760.88822809 1306.90643837  341.67147509  707.07892263
  582.53617771  466.03591462 1252.69938502  466.03591462  582.53617771
  707.07892263  341.67147509 1306.90643837  760.88822809 1034.06483197
  706.03830059  943.92288488 1096.59396683 1276.5770816   318.73741088
 2163.24536665 2181.63500219  462.55791801  655.20455642  545.3760869
 1258.55725821  912.98132691  478.07673821  592.50505948]
[0.33333333 0.33333333 0.33333333]


In [26]:
output_d = compute_d(output,window) 
output_d = output_d[:-2]

In [27]:
result = (output_d>1).astype(np.int8)

In [28]:
result

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
      dtype=int8)

In [29]:
np.save('forward_output.npy',output)
np.save('forward_result.npy',result)

In [30]:
o = np.argsort(-output_d)
o.shape

(42,)

In [34]:
data_time = data_time.to_numpy()
data_time[o]

array(['2022-06-05', '2022-06-20', '2022-04-12', '2022-05-09',
       '2022-05-01', '2022-04-20', '2022-06-06', '2022-06-19',
       '2022-05-24', '2022-05-18', '2022-05-23', '2022-04-19',
       '2022-06-23', '2022-04-14', '2022-04-27', '2022-04-16',
       '2022-05-05', '2022-05-22', '2022-05-13', '2022-05-06',
       '2022-05-08', '2022-05-12', '2022-05-27', '2022-05-04',
       '2022-05-17', '2022-05-07', '2022-05-02', '2022-04-17',
       '2022-06-21', '2022-04-25', '2022-06-22', '2022-06-17',
       '2022-04-30', '2022-04-15', '2022-06-16', '2022-04-13',
       '2022-06-18', '2022-05-26', '2022-04-26', '2022-05-25',
       '2022-05-11', '2022-05-19'], dtype=object)

In [33]:
data_time

88    2022-04-12
87    2022-04-13
86    2022-04-14
85    2022-04-15
84    2022-04-16
83    2022-04-17
82    2022-04-19
81    2022-04-20
80    2022-04-25
79    2022-04-26
78    2022-04-27
77    2022-04-30
76    2022-05-01
75    2022-05-02
74    2022-05-04
73    2022-05-05
72    2022-05-06
71    2022-05-07
70    2022-05-08
69    2022-05-09
68    2022-05-11
67    2022-05-12
66    2022-05-13
65    2022-05-17
64    2022-05-18
63    2022-05-19
62    2022-05-22
61    2022-05-23
60    2022-05-24
59    2022-05-25
58    2022-05-26
57    2022-05-27
56    2022-06-05
55    2022-06-06
54    2022-06-16
53    2022-06-17
52    2022-06-18
51    2022-06-19
50    2022-06-20
49    2022-06-21
48    2022-06-22
47    2022-06-23
Name: time, dtype: object